In [ ]:
# 패키지 수입
import csv
import numpy as np

# natutal language toolkit 
import nltk
#from nltk.corpus import stopwords 
#from nltk.tokenize import word_tokenize 

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from time import time



# 인공신경망 구현시 사용 패키지
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN
from keras.layers import LSTM, Dropout, Embedding
from keras.layers import Bidirectional
from keras.layers import GRU


import pandas as pd
import json

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/NLP/ddd.csv', encoding='UTF-8')

#type(df.name)
df_name = df.name.split(",")

df_name

In [ ]:
df

,Unnamed: 0,Column1,?name,keyword,content,tags,address,address1
0,0,0,"더럭초등학교(구, 더럭분교)",문화,초등학교 동창들과 함께한 가을 여행 속 방문한 애월 더럭분교,"['#제주컬러어택', '#더럭분교', '#초등동창가을여행', '#무지개빛우정']",제주특별자치도 제주시 애월읍 하가로 195,애월읍
1,1,1,"더럭초등학교(구, 더럭분교)",문화,무지개빛 아름다운 더럭초등학교,"['##더럭초등학교', '##더럭분교', '##애월읍']",제주특별자치도 제주시 애월읍 하가로 195,애월읍
2,2,2,"더럭초등학교(구, 더럭분교)",문화,무지개빛 아름다운 더럭초등학교,"['##더럭초등학교', '##더럭분교', '##무지개학교', '##애월읍']",제주특별자치도 제주시 애월읍 하가로 195,애월읍
3,3,3,"더럭초등학교(구, 더럭분교)",문화,알록달록 너무 예쁜 초등학교에서 아내와 함께 좋은 추억과 사진을 남겼어요!,"['##5월엔제주', '##더럭분교', '##더럭초등학교']",제주특별자치도 제주시 애월읍 하가로 195,애월읍
4,4,4,"더럭초등학교(구, 더럭분교)",문화,#제주도 #더럭분교 #연화못 #연꽃마을 정말 뭔가 작은 추억이 자꾸 생각나고 웃음이...,"['#제주도', '#더럭분교', '#연화못', '#연꽃마을']",제주특별자치도 제주시 애월읍 하가로 195,애월읍
...,...,...,...,...,...,...,...,...
11282,11282,14929,서광리 곶자왈생태탐방로,도보,서광리 곶자왈 생태탐방로,[],제주특별자치도 서귀포시 안덕면 서광리,안덕면
11283,11283,14930,제주올레 12코스,도보,제주 올레길 12코스 수월봉 화산쇄설층 정상의 지형은 완만한 편이지만 해안 절벽을 ...,[],제주특별자치도 서귀포시 대정읍 신도리 2883-2,대정읍
11284,11284,14931,제주올레 13코스,도보,올레길 13코스를 꼬닥꼬닥 걸으며... 이날은 클린올레로 올레길을 청소해서 마음도 ...,[],제주특별자치도 제주시 한경면 조수리 2137-2,한경면
11285,11285,14932,제주올레 15-B코스,도보,#올레길 #15-b #담장을 #벗삼아 #태양을 #바라보는 #난제주개,"['#올레길', '#15', '#담장을', '#벗삼아', '#태양을', '#바라보는...",제주특별자치도 제주시 한림읍 한림해안로 192,한림읍


In [ ]:
#df["name"].values    object
#df["category"].values  object
#df["content"].values   object
#df["keyword"].values   object
#df["address"].values    object

type(df)

pandas.core.frame.DataFrame

In [ ]:
df.describe

<bound method NDFrame.describe of        Unnamed: 0  Column1  ...                      address address1
0               0        0  ...      제주특별자치도 제주시 애월읍 하가로 195     애월읍 
1               1        1  ...      제주특별자치도 제주시 애월읍 하가로 195     애월읍 
2               2        2  ...      제주특별자치도 제주시 애월읍 하가로 195     애월읍 
3               3        3  ...      제주특별자치도 제주시 애월읍 하가로 195     애월읍 
4               4        4  ...      제주특별자치도 제주시 애월읍 하가로 195     애월읍 
...           ...      ...  ...                          ...      ...
11282       11282    14929  ...         제주특별자치도 서귀포시 안덕면 서광리     안덕면 
11283       11283    14930  ...  제주특별자치도 서귀포시 대정읍 신도리 2883-2     대정읍 
11284       11284    14931  ...   제주특별자치도 제주시 한경면 조수리 2137-2     한경면 
11285       11285    14932  ...    제주특별자치도 제주시 한림읍 한림해안로 192     한림읍 
11286       11286    14933  ...     제주특별자치도 제주시 추자면 묵리 산 133     추자면 

[11287 rows x 8 columns]>

In [ ]:
df.isnull().sum()

Unnamed: 0    0
Column1       0
?name         0
keyword       0
content       0
tags          0
address       0
address1      0
dtype: int64

In [ ]:
df= df[['content','?name','keyword','address1']]  # 여러개는 대괄호 두번 !!! 잊지말기 

In [ ]:
df

,content,?name,keyword,address1
0,초등학교 동창들과 함께한 가을 여행 속 방문한 애월 더럭분교,"더럭초등학교(구, 더럭분교)",문화,애월읍
1,무지개빛 아름다운 더럭초등학교,"더럭초등학교(구, 더럭분교)",문화,애월읍
2,무지개빛 아름다운 더럭초등학교,"더럭초등학교(구, 더럭분교)",문화,애월읍
3,알록달록 너무 예쁜 초등학교에서 아내와 함께 좋은 추억과 사진을 남겼어요!,"더럭초등학교(구, 더럭분교)",문화,애월읍
4,#제주도 #더럭분교 #연화못 #연꽃마을 정말 뭔가 작은 추억이 자꾸 생각나고 웃음이...,"더럭초등학교(구, 더럭분교)",문화,애월읍
...,...,...,...,...
11282,서광리 곶자왈 생태탐방로,서광리 곶자왈생태탐방로,도보,안덕면
11283,제주 올레길 12코스 수월봉 화산쇄설층 정상의 지형은 완만한 편이지만 해안 절벽을 ...,제주올레 12코스,도보,대정읍
11284,올레길 13코스를 꼬닥꼬닥 걸으며... 이날은 클린올레로 올레길을 청소해서 마음도 ...,제주올레 13코스,도보,한경면
11285,#올레길 #15-b #담장을 #벗삼아 #태양을 #바라보는 #난제주개,제주올레 15-B코스,도보,한림읍


In [ ]:
df.head(15)

,content,?name,keyword,address1
0,초등학교 동창들과 함께한 가을 여행 속 방문한 애월 더럭분교,"더럭초등학교(구, 더럭분교)",문화,애월읍
1,무지개빛 아름다운 더럭초등학교,"더럭초등학교(구, 더럭분교)",문화,애월읍
2,무지개빛 아름다운 더럭초등학교,"더럭초등학교(구, 더럭분교)",문화,애월읍
3,알록달록 너무 예쁜 초등학교에서 아내와 함께 좋은 추억과 사진을 남겼어요!,"더럭초등학교(구, 더럭분교)",문화,애월읍
4,#제주도 #더럭분교 #연화못 #연꽃마을 정말 뭔가 작은 추억이 자꾸 생각나고 웃음이...,"더럭초등학교(구, 더럭분교)",문화,애월읍
5,#겨울 제주 #더럭분교의 추억 #운동장에서 뛰노는 아이들이 부럽다요!!! #오리꽥꽥,"더럭초등학교(구, 더럭분교)",문화,애월읍
6,친구들과 함께 들러서 예쁜 추억 사진 남겼어요. 근처에 연화못도 있어 함꼐 들르기 ...,"더럭초등학교(구, 더럭분교)",문화,애월읍
7,#제주도여행 #고딩친구들 #동심으로 #알록달록 #사진찍기좋은곳 #추억한페이지 #기록...,"더럭초등학교(구, 더럭분교)",문화,애월읍
8,아주오래전 가보기는 한곳이다 가끔지나가기도 하지만 어릴적 국민학교 지금의 초등학교는...,"더럭초등학교(구, 더럭분교)",문화,애월읍
9,#어릴적 시골에서 다니던 국민학교가 생각난 더럭분교 #이곳에서 공부하는 아이들은 참...,"더럭초등학교(구, 더럭분교)",문화,애월읍


In [ ]:
df[:]

,content,?name,keyword,address1
0,초등학교 동창들과 함께한 가을 여행 속 방문한 애월 더럭분교,"더럭초등학교(구, 더럭분교)",문화,애월읍
1,무지개빛 아름다운 더럭초등학교,"더럭초등학교(구, 더럭분교)",문화,애월읍
2,무지개빛 아름다운 더럭초등학교,"더럭초등학교(구, 더럭분교)",문화,애월읍
3,알록달록 너무 예쁜 초등학교에서 아내와 함께 좋은 추억과 사진을 남겼어요!,"더럭초등학교(구, 더럭분교)",문화,애월읍
4,#제주도 #더럭분교 #연화못 #연꽃마을 정말 뭔가 작은 추억이 자꾸 생각나고 웃음이...,"더럭초등학교(구, 더럭분교)",문화,애월읍
...,...,...,...,...
11282,서광리 곶자왈 생태탐방로,서광리 곶자왈생태탐방로,도보,안덕면
11283,제주 올레길 12코스 수월봉 화산쇄설층 정상의 지형은 완만한 편이지만 해안 절벽을 ...,제주올레 12코스,도보,대정읍
11284,올레길 13코스를 꼬닥꼬닥 걸으며... 이날은 클린올레로 올레길을 청소해서 마음도 ...,제주올레 13코스,도보,한경면
11285,#올레길 #15-b #담장을 #벗삼아 #태양을 #바라보는 #난제주개,제주올레 15-B코스,도보,한림읍


#pip install eunjeon

Mecab 설치 https://somjang.tistory.com/entry/Windows-%EC%97%90%EC%84%9C-Mecab-mecab-%EA%B8%B0%EB%B0%98-%ED%95%9C%EA%B5%AD%EC%96%B4-%ED%98%95%ED%83%9C%EC%86%8C-%EB%B6%84%EC%84%9D%EA%B8%B0-%EC%84%A4%EC%B9%98%ED%95%98%EB%8A%94-%EB%B0%A9%EB%B2%95

https://cleancode-ws.tistory.com/97

추후 비교 

In [ ]:
df = df.replace('[^가-힣a-z]', ' ', regex =True)  # 정규 표현식으로 문자열 치환 
df.head()

,content,?name,keyword,address1
0,초등학교 동창들과 함께한 가을 여행 속 방문한 애월 더럭분교,더럭초등학교 구 더럭분교,문화,애월읍
1,무지개빛 아름다운 더럭초등학교,더럭초등학교 구 더럭분교,문화,애월읍
2,무지개빛 아름다운 더럭초등학교,더럭초등학교 구 더럭분교,문화,애월읍
3,알록달록 너무 예쁜 초등학교에서 아내와 함께 좋은 추억과 사진을 남겼어요,더럭초등학교 구 더럭분교,문화,애월읍
4,제주도 더럭분교 연화못 연꽃마을 정말 뭔가 작은 추억이 자꾸 생각나고 웃음이...,더럭초등학교 구 더럭분교,문화,애월읍


In [ ]:
len(df)

11287

In [ ]:
type(df['content'])

pandas.core.series.Series

In [ ]:
df_list = df['content'].values.tolist()

In [ ]:
content = df["content"]

In [ ]:
words = np.concatenate(np.core.defchararray.split(df_list)).tolist()     # df  모든 line 에서 단어 추출해서 하나의 list로 집어넣음
words[0:100]

['초등학교',
 '동창들과',
 '함께한',
 '가을',
 '여행',
 '속',
 '방문한',
 '애월',
 '더럭분교',
 '무지개빛',
 '아름다운',
 '더럭초등학교',
 '무지개빛',
 '아름다운',
 '더럭초등학교',
 '알록달록',
 '너무',
 '예쁜',
 '초등학교에서',
 '아내와',
 '함께',
 '좋은',
 '추억과',
 '사진을',
 '남겼어요',
 '제주도',
 '더럭분교',
 '연화못',
 '연꽃마을',
 '정말',
 '뭔가',
 '작은',
 '추억이',
 '자꾸',
 '생각나고',
 '웃음이',
 '나오는',
 '작은',
 '분교',
 '옆에',
 '활짝',
 '핀',
 '연꽃까지',
 '마음이',
 '깨끗해지는',
 '제주도의',
 '작은',
 '풍경',
 '겨울',
 '제주',
 '더럭분교의',
 '추억',
 '운동장에서',
 '뛰노는',
 '아이들이',
 '부럽다요',
 '오리꽥꽥',
 '친구들과',
 '함께',
 '들러서',
 '예쁜',
 '추억',
 '사진',
 '남겼어요',
 '근처에',
 '연화못도',
 '있어',
 '함꼐',
 '들르기',
 '좋은',
 '거',
 '같습니다',
 '실물보다',
 '사진으로',
 '훨씬',
 '더',
 '아름다운',
 '곳',
 '스냅사진',
 '명소로',
 '유명한',
 '이유가',
 '있었네요',
 '제주도여행',
 '고딩친구들',
 '동심으로',
 '알록달록',
 '사진찍기좋은곳',
 '추억한페이지',
 '기록하기',
 '화려하진',
 '않지만',
 '소박하고',
 '그때',
 '그시절로',
 '돌아간듯한',
 '느낌이라',
 '너무',
 '좋았어요',
 '아주오래전']

In [ ]:
len(words)

112774

In [ ]:
type(words)

list

단어 빈도수 갯수 

In [ ]:
from collections import Counter
result = Counter(words)
result

Counter({'초등학교': 8,
         '동창들과': 1,
         '함께한': 95,
         '가을': 224,
         '여행': 392,
         '속': 20,
         '방문한': 24,
         '애월': 61,
         '더럭분교': 42,
         '무지개빛': 4,
         '아름다운': 423,
         '더럭초등학교': 2,
         '알록달록': 26,
         '너무': 890,
         '예쁜': 179,
         '초등학교에서': 1,
         '아내와': 7,
         '함께': 433,
         '좋은': 292,
         '추억과': 5,
         '사진을': 95,
         '남겼어요': 6,
         '제주도': 1097,
         '연화못': 5,
         '연꽃마을': 1,
         '정말': 451,
         '뭔가': 13,
         '작은': 50,
         '추억이': 34,
         '자꾸': 14,
         '생각나고': 3,
         '웃음이': 3,
         '나오는': 30,
         '분교': 2,
         '옆에': 27,
         '활짝': 8,
         '핀': 12,
         '연꽃까지': 1,
         '마음이': 64,
         '깨끗해지는': 5,
         '제주도의': 72,
         '풍경': 108,
         '겨울': 90,
         '제주': 1247,
         '더럭분교의': 3,
         '추억': 93,
         '운동장에서': 1,
         '뛰노는': 4,
         '아이들이': 73,
         '부럽다요': 1,
    

In [ ]:
result.most_common(100)   # 1차 불용어 판단 

[('제주', 1247),
 ('제주도', 1097),
 ('너무', 890),
 ('수', 684),
 ('제주여행', 583),
 ('있는', 465),
 ('바다', 465),
 ('사려니숲길', 455),
 ('정말', 451),
 ('함께', 433),
 ('아름다운', 423),
 ('여행', 392),
 ('곳', 386),
 ('사려니숲', 365),
 ('성산일출봉', 354),
 ('꼭', 347),
 ('제주가을', 310),
 ('좋은', 292),
 ('더', 284),
 ('그', 271),
 ('제주의', 270),
 ('많이', 269),
 ('카멜리아힐', 233),
 ('가을', 224),
 ('또', 218),
 ('br', 216),
 ('다시', 209),
 ('멋진', 208),
 ('여름', 201),
 ('월', 198),
 ('이', 194),
 ('가족여행', 190),
 ('새별오름', 187),
 ('힐링', 184),
 ('산굼부리', 183),
 ('사려니숲에코힐링체험', 183),
 ('예쁜', 179),
 ('그리고', 175),
 ('한', 172),
 ('제주도여행', 168),
 ('잘', 166),
 ('섭지코지', 166),
 ('우도', 163),
 ('오름', 158),
 ('비짓제주', 155),
 ('것', 148),
 ('가장', 146),
 ('월정리', 146),
 ('있어서', 145),
 ('많은', 145),
 ('사진', 143),
 ('참', 143),
 ('볼', 140),
 ('좋아요', 134),
 ('해변', 133),
 ('jeju', 131),
 ('보고', 130),
 ('다른', 129),
 ('보이는', 129),
 ('첫', 129),
 ('날', 128),
 ('산과오름', 128),
 ('다', 125),
 ('할', 121),
 ('물찻오름', 121),
 ('협재해수욕장', 121),
 ('일', 119),
 ('월정리해변', 119),
 ('년',

In [ ]:
# br =  html 태그인것 같아 삭제 
type(words)
#words
#words = words.replace('br' ,'')

result_words = []

for i in words:
  temp = i.replace('br' ,'')
  result_words.append(temp)
print(result_words)

['초등학교', '동창들과', '함께한', '가을', '여행', '속', '방문한', '애월', '더럭분교', '무지개빛', '아름다운', '더럭초등학교', '무지개빛', '아름다운', '더럭초등학교', '알록달록', '너무', '예쁜', '초등학교에서', '아내와', '함께', '좋은', '추억과', '사진을', '남겼어요', '제주도', '더럭분교', '연화못', '연꽃마을', '정말', '뭔가', '작은', '추억이', '자꾸', '생각나고', '웃음이', '나오는', '작은', '분교', '옆에', '활짝', '핀', '연꽃까지', '마음이', '깨끗해지는', '제주도의', '작은', '풍경', '겨울', '제주', '더럭분교의', '추억', '운동장에서', '뛰노는', '아이들이', '부럽다요', '오리꽥꽥', '친구들과', '함께', '들러서', '예쁜', '추억', '사진', '남겼어요', '근처에', '연화못도', '있어', '함꼐', '들르기', '좋은', '거', '같습니다', '실물보다', '사진으로', '훨씬', '더', '아름다운', '곳', '스냅사진', '명소로', '유명한', '이유가', '있었네요', '제주도여행', '고딩친구들', '동심으로', '알록달록', '사진찍기좋은곳', '추억한페이지', '기록하기', '화려하진', '않지만', '소박하고', '그때', '그시절로', '돌아간듯한', '느낌이라', '너무', '좋았어요', '아주오래전', '가보기는', '한곳이다', '가끔지나가기도', '하지만', '어릴적', '국민학교', '지금의', '초등학교는', '확연히', '분명다르다', '내부와', '외부', '특히', '외부에', '칠해진', '무지개', '빛깔', '색들이', '잔디색과', '아주', '잘', '어울리는', '곳이다', '또한', '내부', '시설과', '아이들의', '미소', '가', '아주', '보기좋았다', '우연히', '보게된', '풍물놀이패라고나', '할까', '여러장구를', '동시에', '치는', 

from collections import Counter

result2 = Counter(result_words)

result2.most_common(100)

br을 공백으로 대체확인

In [ ]:
pip install konlpy

     |████████████████████████████████| 19.4 MB 1.3 MB/s 
     |████████████████████████████████| 448 kB 55.2 MB/s 
     |████████████████████████████████| 86 kB 5.7 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [ ]:
# 형태소 분석(noun기준) 함수 구성 
from konlpy.tag import Okt
import re
okt = Okt()
res = []
for str_i in content[:] : 
  tk = okt.nouns(str_i )   # 확인
  res.append(tk)
res

[['초등학교', '동창', '가을', '여행', '속', '방문', '애월', '더럭', '분교'],
 ['무지개', '빛', '더럭', '초등학교'],
 ['무지개', '빛', '더럭', '초등학교'],
 ['알록달록', '초등학교', '아내', '추억', '사진'],
 ['제주도',
  '더럭',
  '분교',
  '연화',
  '못',
  '연꽃',
  '마을',
  '정말',
  '뭔가',
  '추억',
  '자꾸',
  '웃음',
  '분교',
  '옆',
  '활짝',
  '핀',
  '연꽃',
  '마음',
  '제주도',
  '풍경'],
 ['겨울', '제주', '더럭', '교의', '추억', '운동장', '아이', '오리'],
 ['친구',
  '추억',
  '사진',
  '근처',
  '연화',
  '못',
  '거',
  '실물',
  '사진',
  '더',
  '곳',
  '스냅',
  '사진',
  '명소',
  '이유'],
 ['제주도',
  '여행',
  '고딩',
  '친구',
  '동심',
  '알록달록',
  '사진',
  '찍기',
  '곳',
  '추억',
  '페이지',
  '기록',
  '그때',
  '시절',
  '느낌'],
 ['아주',
  '곳',
  '가끔',
  '적',
  '국민학교',
  '지금',
  '초등학교',
  '확',
  '분명',
  '내부',
  '외부',
  '외부',
  '무지개',
  '빛깔',
  '색',
  '잔디',
  '색',
  '아주',
  '곳',
  '또한',
  '내부',
  '시설',
  '아이',
  '미소',
  '아주',
  '보기',
  '풍물놀이',
  '패',
  '나',
  '장구',
  '동시',
  '학생',
  '연습',
  '공연',
  '아주',
  '수준'],
 ['적',
  '시골',
  '국민학교',
  '더럭',
  '분교',
  '곳',
  '공부',
  '아이',
  '공부',
  '맛',
  '집',
  '그',
  '여기',
  '생각

In [ ]:
# 분석된 형태소를 하나의 list로 통합
res2 = []
for i in range(len(res)) :
    ok = Counter(res[i])
    res2.extend(ok)
res2

['초등학교',
 '동창',
 '가을',
 '여행',
 '속',
 '방문',
 '애월',
 '더럭',
 '분교',
 '무지개',
 '빛',
 '더럭',
 '초등학교',
 '무지개',
 '빛',
 '더럭',
 '초등학교',
 '알록달록',
 '초등학교',
 '아내',
 '추억',
 '사진',
 '제주도',
 '더럭',
 '분교',
 '연화',
 '못',
 '연꽃',
 '마을',
 '정말',
 '뭔가',
 '추억',
 '자꾸',
 '웃음',
 '옆',
 '활짝',
 '핀',
 '마음',
 '풍경',
 '겨울',
 '제주',
 '더럭',
 '교의',
 '추억',
 '운동장',
 '아이',
 '오리',
 '친구',
 '추억',
 '사진',
 '근처',
 '연화',
 '못',
 '거',
 '실물',
 '더',
 '곳',
 '스냅',
 '명소',
 '이유',
 '제주도',
 '여행',
 '고딩',
 '친구',
 '동심',
 '알록달록',
 '사진',
 '찍기',
 '곳',
 '추억',
 '페이지',
 '기록',
 '그때',
 '시절',
 '느낌',
 '아주',
 '곳',
 '가끔',
 '적',
 '국민학교',
 '지금',
 '초등학교',
 '확',
 '분명',
 '내부',
 '외부',
 '무지개',
 '빛깔',
 '색',
 '잔디',
 '또한',
 '시설',
 '아이',
 '미소',
 '보기',
 '풍물놀이',
 '패',
 '나',
 '장구',
 '동시',
 '학생',
 '연습',
 '공연',
 '수준',
 '적',
 '시골',
 '국민학교',
 '더럭',
 '분교',
 '곳',
 '공부',
 '아이',
 '맛',
 '집',
 '그',
 '여기',
 '생각',
 '대학',
 '동기',
 '베스트',
 '프렌드',
 '제주',
 '여행',
 '더럭',
 '분교',
 '하늘',
 '전부',
 '생각',
 '만해',
 '추억',
 '미소',
 '남자친구',
 '처음',
 '여행',
 '저기',
 '계획',
 '짯는데',
 '역시',
 '맛',
 '박',
 '다음',
 '일주일

In [ ]:
# 최다빈출 단어 조사 후 2차 불용어 판단
Count_res = Counter(res2)
Count_res.most_common(100) 

[('제주', 3034),
 ('여행', 1977),
 ('제주도', 1563),
 ('곳', 1456),
 ('바다', 1058),
 ('해변', 1046),
 ('가을', 787),
 ('오름', 781),
 ('사려', 780),
 ('사진', 731),
 ('힐링', 693),
 ('숲길', 650),
 ('월', 626),
 ('수', 561),
 ('가족', 542),
 ('숲', 540),
 ('성산일출봉', 464),
 ('날', 431),
 ('정말', 426),
 ('시간', 412),
 ('아이', 392),
 ('체험', 381),
 ('때', 377),
 ('풍경', 374),
 ('꼭', 361),
 ('여름', 357),
 ('바람', 349),
 ('추억', 340),
 ('더', 333),
 ('것', 313),
 ('정리', 306),
 ('해수욕장', 301),
 ('자연', 299),
 ('또', 291),
 ('겨울', 290),
 ('친구', 288),
 ('코스', 288),
 ('함덕', 286),
 ('힐', 284),
 ('에코', 280),
 ('생각', 279),
 ('카멜리아', 279),
 ('날씨', 278),
 ('길', 269),
 ('사람', 268),
 ('그', 259),
 ('공원', 259),
 ('년', 254),
 ('우도', 251),
 ('추천', 248),
 ('하늘', 244),
 ('마음', 242),
 ('다시', 242),
 ('억새', 242),
 ('나', 235),
 ('새별오름', 235),
 ('비', 234),
 ('사랑', 234),
 ('우리', 225),
 ('산굼부리', 223),
 ('처음', 221),
 ('해안', 213),
 ('모습', 212),
 ('산책', 212),
 ('밭', 209),
 ('이', 208),
 ('혼자', 207),
 ('방문', 204),
 ('섭지코지', 204),
 ('빛', 199),
 ('산', 198),
 ('엄마'

In [ ]:
   # 2차 확인

# 불용어 제거 패키지 다운로드
import nltk
nltk.download('stopwords')
nltk.download('punkt')

# 영어불용어 제거! 
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 


example = df["content"]

stopwords = set(stopwords.words('english'))

word_tokens = word_tokenize((example)

result = []
for w in word_tokens :
    if w in stopwords:
        result.append(w)

print(word_tokens)
print(result)

In [1]:
#불용어 리스트 작성
stopwords = '너무 수 있는 함께 정말 곳 더 의 많이 또 이 한 다시 참 그리고 보고 것 많은 가장 보이는 날 있어서 년 같아요 할 중 다 본 첫 하고 있어 일 있고 다른 있다 있습니다 너무나 월 수 날 정말 때 꼭 더 것 또 길 그 년 다시 나 이 봉 보고 중 한번 일 눈 속 광 저 분'.split() 
stopwords

['너무',
 '수',
 '있는',
 '함께',
 '정말',
 '곳',
 '더',
 '의',
 '많이',
 '또',
 '이',
 '한',
 '다시',
 '참',
 '그리고',
 '보고',
 '것',
 '많은',
 '가장',
 '보이는',
 '날',
 '있어서',
 '년',
 '같아요',
 '할',
 '중',
 '다',
 '본',
 '첫',
 '하고',
 '있어',
 '일',
 '있고',
 '다른',
 '있다',
 '있습니다',
 '너무나',
 '월',
 '수',
 '날',
 '정말',
 '때',
 '꼭',
 '더',
 '것',
 '또',
 '길',
 '그',
 '년',
 '다시',
 '나',
 '이',
 '봉',
 '보고',
 '중',
 '한번',
 '일',
 '눈',
 '속',
 '광',
 '저',
 '분']

In [2]:
# 불용어 제거 패키지 다운로드
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
# 불용어 제거 
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

stopwords = stopwords
tok = word_tokenize(str(res2))

def stopword(word_tokenize):
  result = []


  for w in word_tokenize :
    if w not in stopword:
      result.append(w)
  return result
    
#  stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다',"와","너무","정말","있는","함께","곳","꼭","그","더","많이","br","뭘","또"
#,"다시","그리고","것","참","보고","많은" ]
#  token_list = []

#  for text in token_list :
#    token = Tokenizer.nouns(res)
#    token = [t for t in token if t not in stopwords or type(t) != float]
#    token_list.append(token)
#  return token_list

NameError: ignored

In [ ]:
tok  

In [ ]:
len(tok)

In [ ]:
check = []

for i in tok:
  temp = i.replace(",","")
  check.append(temp)
print(check)


In [ ]:
check.remove("'")
check

In [ ]:
check.remove( "'")

In [ ]:
check

### vectorizer 기초코드 작성

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
vectors = vectorizer.fit_transform(tokens) #여러 개의 문장을 넣어줘야 작동합니다!!

print(vectorizer.get_feature_names())
print(vectors.toarray())

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
vectors = vectorizer.fit_transform(tokens) #여러 개의 문장을 넣어줘야 작동합니다!!

print(vectorizer.get_feature_names())
print(vectors.toarray())

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(min_df=0)
tfidf_vectorizer = tfidf.fit_transform(tokens) 

#tf-idf dictionary    
tfidf_dict = tfidf.get_feature_names()
print(tfidf_dict)
print(tfidf_vectorizer.toarray())

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(min_df=0)
tfidf_vectorizer = tfidf.fit_transform(tokens) 

#tf-idf dictionary    
tfidf_dict = tfidf.get_feature_names()
print(tfidf_dict)
print(tfidf_vectorizer.toarray())